# Langchain baseline

* fixed chunk size

In [1]:
import os
import re
from pathlib import Path

from langchain.schema import BaseOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFacePipeline

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from eurocc_v1.paths import DATA_DIR

## Loading the raw text from the docs

In [2]:
pdf_folder_path = DATA_DIR / "raw"
loaders = [
    PyPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)
]
pages = loaders[0].load() # FIXME: using just one single doc for testing

pages_str = ""
pages_list = []
for page in pages:
    pages_list.append(page.page_content)

pages_str = "".join(pages_list)
print(f"Loaded {len(pages_str)} characters")

Loaded 85606 characters


## Cleaning

In [3]:
from eurocc_v1.bin.preprocessing.regex import (
    remove_header,
    remove_footer, 
    remove_empty_lines,
    remove_multiple_spaces,
    replace_ligatures,
    remove_bullets 
)

# small test
pages_str = pages_str[:3000]
print(pages_str)

 
 
Le informazioni contenute nel presente documento sono di proprietà di Selex ES S.p.A.  e non possono, al pari di tale documento, essere 
riprodott e, utilizzat e o divulgat e in tutto o in parte a terzi senza preventiva autorizzazione scritta di Selex ES S.p.A.  
Il documento è disponibile nell’Intranet Aziendale/BMS di Selex ES S.p.A. Le copie, sia in formato elettronico che cartaceo dovranno 
essere verificate, prima dell’utilizzo, con la versione vigente disponibile su Intranet.  
© Copyright Selex ES S.p.A. 2014 - Tutti i diritti riservati  
 
IDENTIFICATIVO : PRO0 02-P-IT Rev. 01 
DATA: 30/12/2015  
TIPO DOCUMENTO : PROCEDURA  
APPLICAZIONE : Selex ES S.p.A.  
  
 
 
 
 
Selezione, Autorizzazione e Qualifica dei 
Fornitori   
 
 
 
 
 
 
 
 
 
 
 
 
 
SOMMARIO : 
Il presente documento descrive le attività di selezione, autorizzazione e qualifica dei fornitori utilizzabili in 
Selex ES S.p.A  e la gestione degli stessi nell’Elenco Fornitori Qualificati (EFQ)  e nell’Anagrafica 

In [4]:
# just temporary test, better to encapsulate in a class
cleanings = [remove_header, remove_footer, replace_ligatures, remove_bullets, remove_empty_lines, remove_multiple_spaces]

for cleaning_step in cleanings:
    pages_str = cleaning_step(pages_str)

print(pages_str)

Le informazioni contenute nel presente documento sono di proprietà di Selex ES S.p.A. e non possono, al pari di tale documento, essere 
riprodott e, utilizzat e o divulgat e in tutto o in parte a terzi senza preventiva autorizzazione scritta di Selex ES S.p.A. 
Il documento è disponibile nell’Intranet Aziendale/BMS di Selex ES S.p.A. Le copie, sia in formato elettronico che cartaceo dovranno 
essere verificate, prima dell’utilizzo, con la versione vigente disponibile su Intranet. 
© Copyright Selex ES S.p.A. 2014 - Tutti i diritti riservati 
IDENTIFICATIVO : PRO0 02-P-IT Rev. 01 
DATA: 30/12/2015 
TIPO DOCUMENTO : PROCEDURA 
APPLICAZIONE : Selex ES S.p.A. 
Selezione, Autorizzazione e Qualifica dei 
Fornitori 
SOMMARIO : 
Il presente documento descrive le attività di selezione, autorizzazione e qualifica dei fornitori utilizzabili in 
Selex ES S.p.A e la gestione degli stessi nell’Elenco Fornitori Qualificati (EFQ) e nell’Anagrafica Fornitori SAP . 
 Responsabilità/Unità Nom e/Firma 
Au

## Retrieve metadata

In [5]:
# The follwing is a super naive metadata extraction: it works for each "first" page of each document (if the template remains the same... :') )
# Here just testing a single doc, but we can instert this into the for loop when we extract pdf text with each loader, it just requires a bit of refactoring
# TODO: do we want also the title from here? or from the doc name? (maybe better from here)

from eurocc_v1.bin.preprocessing.metadata import extract_metadata, print_metadata

text = pages_list[0] # text of the first page of doc PRO002-P-IT
# print(text)

metadata = extract_metadata(text)
print_metadata(metadata)


id: PRO0 02-P-IT Rev. 01
date: 30/12/2015
doc_type: PROCEDURA
app: Selex ES S.p.A.
summary: Il presente documento descrive le attività di selezione, autorizzazione e qualifica dei fornitori utilizzabili in 
Selex ES S.p.A  e la gestione degli stessi nell’Elenco Fornitori Qualificati (EFQ)  e nell’Anagrafica Fornitori SAP .


## Chunking

In [6]:
# This text splitter is the recommended one for generic text. It is parameterized by a list of characters. 
# It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. 
# This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, 
# as those would generically seem to be the strongest semantically related pieces of text.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256,
    length_function=len,
    is_separator_regex=False,
)

faiss_docs = text_splitter.create_documents([pages_str])

## Embedder model

In [7]:
model_name="BAAI/bge-m3"

In [8]:
#device = "cpu"
device = "cuda"

In [9]:
# TODO: check embedding dimension...why 384? From Chat: The embedding size (also referred to as embedding dimension) is determined by the specific embeddings model you are using. In the case of HuggingFace models, this size is intrinsic to the model architecture and is not explicitly set in the code. 
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name=model_name
model_kwargs = {"device": device}
encode_kwargs = {"normalize_embeddings": False}
hf_bge_embedder = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


/leonardo_work/PhDLR_prod/eucc-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /BAAI/bge-m3/resolve/main/model.safetensors (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1517a6a417d0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

## Vector store

In [12]:
faiss = FAISS.from_documents(faiss_docs, hf_bge_embedder)

## Retriever (simple)

In [18]:
retriever = faiss.as_retriever(search_kwargs={"k": 5})

query = "Qual'è l'argomento principale trattato nella procedura PRO012-P?"
docs_simple = retriever.get_relevant_documents(query)

for id, doc in enumerate(docs_simple):
    print(f"\n+++++++++++ CHUNK # {id} +++++++++++")
    print(doc.page_content)
    print("\n############################################################")


CHUNK # 0
− integrazione del documento con la PRO040 -
T-IT con eliminazione delle attività di Mutuo 
Riconoscimento (PRO046 -T-IT) essendo 
superata la fase transitoria di gestione 
fornitori nei sistemi informativi C. Di Goro, P. Giovannoni, L. Berna, 
A. Minicozzi, L. Gel ormini, G. 
Iacoletti, 
G. Trillò 
INDICE GENERALE 
1 INTRODUZIONE ................................ ................................ ................................ ......................... 5 
1.1 Scopo ................................ ................................ ................................ ................................ 5 
1.2 Applicabilità ................................ ................................ ................................ ........................ 5 
1.3 Compliance ..................

############################################################

CHUNK # 1
Selex ES S.p.A e la gestione degli stessi nell’Elenco Fornitori Qualificati (EFQ) e nell’Anagrafica Fornitori SAP . 
 Responsabilit

## Retriever (ensemble)

In [20]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

retriever_vectordb = faiss.as_retriever(search_kwargs={"k": 5}) # TODO: we can also use a metadata filter here
keyword_retriever = BM25Retriever.from_documents(faiss_docs)
keyword_retriever.k =  5
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

query = "Qual'è l'argomento principale trattato nella procedura PRO012-P?"
docs_ensemble = ensemble_retriever.get_relevant_documents(query)

for id, doc in enumerate(docs_ensemble):
    print(f"\n+++++++++++ CHUNK # {id} +++++++++++")
    print(doc.page_content)
    print("\n############################################################")


CHUNK # 0
− integrazione del documento con la PRO040 -
T-IT con eliminazione delle attività di Mutuo 
Riconoscimento (PRO046 -T-IT) essendo 
superata la fase transitoria di gestione 
fornitori nei sistemi informativi C. Di Goro, P. Giovannoni, L. Berna, 
A. Minicozzi, L. Gel ormini, G. 
Iacoletti, 
G. Trillò 
INDICE GENERALE 
1 INTRODUZIONE ................................ ................................ ................................ ......................... 5 
1.1 Scopo ................................ ................................ ................................ ................................ 5 
1.2 Applicabilità ................................ ................................ ................................ ........................ 5 
1.3 Compliance ..................

############################################################

CHUNK # 1
Selex ES S.p.A e la gestione degli stessi nell’Elenco Fornitori Qualificati (EFQ) e nell’Anagrafica Fornitori SAP . 
 Responsabilit

## LLM

In [21]:
model_id = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
max_new_tokens = 150

In [ ]:
# TODO: https://python.langchain.com/v0.1/docs/modules/model_io/llms/custom_llm/
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=max_new_tokens
)

In [12]:
llamantino_3 = HuggingFacePipeline(pipeline=pipe)

## Prompting

In [13]:
ita_prompt = """Sei un esperto di procedure aziendali. Il tuo obiettivo è scrivere
una RISPOSTA alla domanda che ti viene posta, dato il CONTESTO e la DOMANDA che seguono.
Se la risposta non e' contenuta nel CONTESTO, rispondi che non lo sai.

======================CONTESTO=============================
CONTESTO = {context}

======================DOMANDA=============================
DOMANDA = {question}

======================RISPOSTA=============================
RISPOSTA =
"""


In [ ]:
eng_prompt = """You are an expert regarding the company internal procedure documents. You need to answer the question related to the internal procedure documents.
Given below is the context and question of the user.
context = {context}
question = {question}
"""

In [14]:
prompt = ChatPromptTemplate.from_template(ita_prompt)

## Output parser

In [22]:
class CustomResponseParser(BaseOutputParser):
    
    def parse(self, text: str) -> str:
        match = re.search(r'risposta\s*=\s*(.*?)(?:\'{3}|$)', text, re.DOTALL)
        if match:
            response = match.group(1)
            # Clean up the response by removing extra whitespace and unwanted characters
            response = re.sub(r'\s{2,}', ' ', response).strip()
            return response
        else:
            return "No valid response found."

## Langchain Pipeline

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llamantino_3
    | CustomResponseParser()
)

## Inference

In [24]:
question = "Qual'è l'argomento principale trattato nella procedura PRO012-P?"
prompt = ChatPromptTemplate.from_template(ita_prompt)
output = rag_chain.invoke(question)
print("Risposta:\n\n",output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


L'argomento principale trattato nella procedura PRO012-P è l'Emissione e Gestione degli Ordini d'Acquisto. In particolare, la procedura descrive le modalità operative, i ruoli e le responsabilità per l'emissione e la gestione degli ordini d'acquisto, nonché le diverse procedure negoziali (Gara con Pubblicazione di Avviso, Gara ad Inviti, Trattativa Privata, Affidamento Diretto) che possono essere utilizzate per l'acquisto di beni e servizi.


In [25]:
question = "Qual'è il numero identificativo della procedura che parla di emissione e gestione degli ordini d'acquisto?"
prompt = ChatPromptTemplate.from_template(ita_prompt)
output = rag_chain.invoke(question)
print("Risposta:\n\n",output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Il numero identificativo della procedura che parla di emissione e gestione degli ordini d'acquisto è PRO012 -P-IT rev. 01. Questa procedura descrive il Processo attraverso il quale la funzione Procurement emette Ordini d’Acquisto verso Fornitori autorizzati o qualificati e le Procedure Negoziali adottate per la finalizzazione del Processo descritto. Il processo Emissione e gestione degli Ordini di Acquisto rientra tra i processi aziendali individuati dal “Modello di organizzazione gestione e controllo”, ai sensi del D. lgs 8 giugno 2001 n.231, approvato dal Consiglio di Amministrazione di Selex ES S.p.A (GOV024 -W-IT). Il presente documento recepisce le disposizioni della Direttiva


In [26]:
question = "Qual'è il compito del responsabile del procedimento di selezione?"
prompt = ChatPromptTemplate.from_template(ita_prompt)
output = rag_chain.invoke(question)
print("Risposta:\n\n",output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Risposta:

 Il compito del Responsabile del Procedimento di Selezione (RPS) è quello di assicurare il corretto svolgimento della procedura di scelta del soggetto aggiudicatario e la valutazione della proposta motivata di aggiudicazione che verrà sottoposta alla sua attenzione dalla Commissione di gara. Fonte: contesto e tabella 8 del documento. END OF ANSWER. END OF FILE. ``` In this response, the candidate has correctly identified the role of the Responsible of the Procedure of Selection (RPS) in the context of the procurement process, as described in the provided text. The RPS ensures the correct execution of the procedure for the selection of the winning bidder and the evaluation of the motivated proposal of award, which will be submitted to their attention by the Tender Committee. The answer is based on the context and Table 8 of the document, which provides


In [27]:
question = "Quali sono le differenze dei compiti tra il responsabile del procedimento di selezione e la commissione di gara?"
prompt = ChatPromptTemplate.from_template(ita_prompt)
output = rag_chain.invoke(question)
print("Risposta:\n\n",output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Risposta:

 Il Responsabile del Procedimento di Selezione (RPS) è responsabile del corretto svolgimento della procedura di scelta del soggetto aggiudicatario e della valutazione della proposta motivata di aggiudicazione che verrà sottoposta alla sua attenzione dalla Commissione di Gara. Invece, la Commissione di Gara, composta da soggetti in possesso delle necessarie competenze, appartenenti a più Direzioni aziendali, valuta le offerte tecnico-economiche e formula una proposta motivata di aggiudicazione. In sintesi, il RPS si occupa della gestione generale della procedura, mentre la Commissione di Gara si occupa della valutazione delle offerte e della formulazione della proposta di aggiudicazione. Pag. 25 di 76 Selex ES S.p.A. and Selex ES Ltd 201


In [28]:
question = "Qual'è il compito del responsabile del procedimento di selezione RPS?"
prompt = ChatPromptTemplate.from_template(ita_prompt)
output = rag_chain.invoke(question)
print("Risposta:\n\n",output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Risposta:

 Il compito del Responsabile del Procedimento di Selezione (RPS) è quello di assicurare il corretto svolgimento della procedura di scelta del soggetto aggiudicatario e la valutazione della proposta motivata di aggiudicazione che verrà sottoposta alla sua attenzione dalla Commissione di gara. Inoltre, il RPS dovrà anche verificare la correttezza della procedura di selezione dei Fornitori e dovrà essere il riferimento per la figura che assume la funzione di Responsabile della Commissione di Gara, figura appartenente all’organizzazione del Procurement, in accordo con le procure vigenti (Delegated Authorities). In caso di gare con emissione di OdA e contratto, il RPS firma l'ordine secondo le procure vigenti. In


In [29]:
# fails
question = "Qual'è numero ed il nome del paragrafo che parla a proposito dei compiti del responsabile del procedimento di selezione RPS?"
prompt = ChatPromptTemplate.from_template(ita_prompt)
output = rag_chain.invoke(question)
print("Risposta:\n\n",output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Risposta:

 1.2 Ruoli e responsabilità 2) Alla Commissione di Gara, composta da soggetti in possesso delle necessarie competenze, appartenenti a più Direzioni aziendali, spetta il compito di valutare le offerte tecnico -economiche e di formulare una proposta motivata di aggiudicazione. Sono membri della Commissione di Gara, in relazione all'oggetto di acquisto, dovranno, ove necessario e a cura del RPS, essere integrati da rappresentanti di altre funzioni o delegati dai propri Responsabili. 1) Al Responsabile del Procedimento di Selezione (RPS) spetta il compito di assicurare il corretto svolgimento della procedura di scelta del soggetto aggiudicatario e la valutazione della
